<a href="https://colab.research.google.com/github/paulcodrea/reddit_humor/blob/main/Humour_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Get reddit /r/dadjokes data
import requests

url = "https://www.reddit.com/r/dadjokes.json"
resp = requests.get(url=url, headers = {'User-agent': 'Humour detection bot'}, params = {'limit': '100'})
data = resp.json()
data_list = []
data_list.append(data)
# repeat process 9 more times with updated 'after' param
for i in range(9):
  after_param = data['data']['after'] 
  data = requests.get(url=url, headers = {'User-agent': 'Humour detection bot'}, params = {'after': after_param, 'limit': '100'})
  data_2 = data.json()
  data_list.append(data_2)
  data = data_2

In [7]:
# create a list of all jokes in json format
data = []
for collection in data_list:
  collection_list = collection['data']['children']
  for joke_index in range(len(collection_list)):
    data.append(collection_list[joke_index]['data'])

In [8]:
# Data set:
# joke 0
print(data[0])
# joke 0 title
print(data[0]['title'])
# joke 0 punchline
print(data[0]['selftext'])
# joke 0 up votes
print(data[0]['ups'])
# joke 0 down votes 
print(data[0]['downs'])
# joke 0 up votes ratio - what is that?
print(data[0]['upvote_ratio'])
# we can also use awards 
print(data[0]['total_awards_received'])


{'approved_at_utc': None, 'subreddit': 'dadjokes', 'selftext': 'I was being transparent.', 'author_fullname': 't2_15rd4c', 'saved': False, 'mod_reason_title': None, 'gilded': 2, 'clicked': False, 'title': 'My kid came out to me as trans and asked if I still accepted them for who they are. I told them quite clearly that I loved them no matter what they chose.', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/dadjokes', 'hidden': False, 'pwls': 6, 'link_flair_css_class': None, 'downs': 0, 'thumbnail_height': 0, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_u7guwu', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 0.87, 'author_flair_background_color': None, 'subreddit_type': 'public', 'ups': 10135, 'total_awards_received': 14, 'media_embed': {}, 'thumbnail_width': 0, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': [], 'secure_media': None, 'is_reddit_media_domain': False, 'is_meta': False, 'category': '', 'secure_